<a href="https://colab.research.google.com/github/corinnakeum1/DS2002Project2/blob/main/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
weather_df = pd.read_csv('weather_data.csv')

                                                 7  1.191968116  25.54900443
San Antonio  4/29/2024 13:35  37.837682  76.626202     3.465576    16.288698
Houston      2/27/2024 15:56 -3.259009   39.769348     7.411353    16.650839
New York     1/10/2024 6:42  -9.098840   80.051711     5.526277    28.612714
San Antonio  1/29/2024 18:07  32.587341  34.421484     7.429063    29.314494
Philadelphia 3/6/2024 14:30   28.966143  47.730579     8.081970     9.868444


In [ ]:
# NEW  CODE FOR APP.PY ON 5/9

from flask import Flask, render_template, request, jsonify
import requests
import string
import re

app = Flask(__name__)

weather_df = pd.read_csv('weather_data.csv')

cities_in_csv = ["Chicago", "Dallas", "Houston", "Los Angeles", "New York", "Philadelphia", "Phoenix", "San Antonio", "San Diego", "San Jose"]

# WEATHER CSV FUNCTION
def get_avg_weather(city):
    df_city = weather_df[weather_df['Location'].str.lower() == city.lower()]

    if df_city.empty:
        return f"Sorry, no data available for {city.title()}."

    temp = df_city['Temperature_C'].mean()
    humidity = df_city['Humidity_pct'].mean()
    precipitation = df_city['Precipitation_mm'].mean()
    wind_speed = df_city['Wind_Speed_kmh'].mean()

    return (f"Average weather for {city.title()} based on local records:\n"
            f"- Temperature: {temp:.2f}°C\n"
            f"- Humidity: {humidity:.1f}%\n"
            f"- Precipitation: {precipitation:.2f} mm\n"
            f"- Wind Speed: {wind_speed:.2f} km/h")

# WEATHER API FUNCTION
def get_live_weather(city):
    api_key = "REDACTED"
    url = f"http://api.weatherstack.com/current?access_key={api_key}&query={city}&air_quality=1"

    try:
        response = requests.get(url)
        if response.status_code != 200:
            return f"Error: Could not reach the weather API."

        data = response.json()

        if "location" not in data or not data["location"]:
            return f"Sorry, I couldn't find weather info for that."

        # Extract relevant weather info
        location = data["location"]["name"]
        country = data["location"]["country"]
        temp = data["current"]["temperature"]
        condition = data["current"]["weather_descriptions"][0]
        feels_like = data["current"]["feelslike"]
        humidity = data["current"]["humidity"]
        moon_phase = data["current"].get("astro", {}).get("moon_phase", "unknown")

        return (f"The weather in {location}, {country} is currently {condition.lower()} with "
                f"a temperature of {temp}°C (feels like {feels_like}°C) and humidity at {humidity}%. "
                f"Moon phase is {moon_phase.lower()}.")

    except Exception as e:
        return f"An error occurred: {e}"

# ROUTES
@app.route('/')
def home():
    return render_template('home.html')

@app.route('/debug')
def debug():
    return "This is the debug route. Everything's (hopefully) fine."

@app.route('/about')
def about():
    return "This is a simple Flask app."

@app.route('/chat', methods=['POST'])
def chat():
    user_input = request.json.get('message', '')
    check_city = next((city for city in cities_in_csv if city in user_input), None)
    if "weather" in user_input.lower():
        if ("average" in user_input.lower() or "past" in user_input.lower() or "hist" in user_input.lower() or "was" in user_input.lower()) and check_city:
            city = extract_city(user_input)
            response = get_avg_weather(city)
        elif "current" in user_input.lower() or "is" in user_input.lower() or "now" in user_input.lower():
            city = extract_city(user_input)
            response = get_live_weather(city)
        else:
            response = "Hmm, I'm not sure. Please check your spelling and capitalization, and ask me again!"
    else:
        response = "Try asking about the weather in a city."

    return jsonify({'response': response})

# UTIL FUNCTION
def extract_city(user_input):
    # Remove punctuation like '?'
    user_input = user_input.replace('?', '').strip()

    # Regex to match capitalized words (this will capture each capitalized word in the sentence)
    capitalized_words = re.findall(r'\b[A-Z][a-z]*\b(?:\s[A-Z][a-z]*)*', user_input)

    # List of words to remove from capitalized words
    remove_words = {"What", "Tell", "Can", "Average", "Current", "Weather"}

    # Remove non-city capitalized words
    refined_words = [word for word in capitalized_words if word not in remove_words]

    # Put together cities longer than one word
    if len(refined_words) > 1:
        city_name = ' '.join(refined_words)
    elif len(capitalized_words) == 1:
        # Handles one word cities
        city_name = refined_words[0]
    else:
        city_name = "Invalid"

    return city_name

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [ ]:
# HOME.HTML FROM 5/9

<!DOCTYPE html>
<html>
<head>
    <title>Home Page</title>
</head>
<body>
    <h1>Weather Chatbot</h1>
    <p>This is the home page.</p>
    <a href="/about">About</a> |
    <a href="/debug">Debug</a>

    <h2>Ask about historical (average) or current weather data in a city.</h2>
    <div id="chatbox"></div>
    <input type="text" id="message" placeholder="Ask something..." />
    <button onclick="sendMessage()">Send</button>

    <script>
        async function sendMessage() {
            const inputBox = document.getElementById("message");
            const chatBox = document.getElementById("chatbox");
            const userMessage = inputBox.value;

            if (!userMessage.trim()) return;

            chatBox.innerHTML += "<p><b>You:</b> " + userMessage + "</p>";
            inputBox.value = "";

            const res = await fetch("/chat", {
                method: "POST",
                headers: {"Content-Type": "application/json"},
                body: JSON.stringify({message: userMessage})
            });

            const data = await res.json();
            chatBox.innerHTML += "<p><b>Bot:</b> " + data.response + "</p>";
        }
    </script>
</body>
</html>